# Test RooFit constant caching using formula PDFs

In [1]:
import ROOT
ROOT.gROOT.ProcessLine(".x $HSCODE/hsfit/LoadFit.C")
%jsroot

Welcome to JupyROOT 6.16/00


create toy manager to generate 1 data set

In [2]:
toy = ROOT.ToyManager(1)
toy.SetUp().SetOutDir("outOptimise/");

Declare 1 fit variable, an azimuthal angle Phi

In [3]:
toy.SetUp().LoadVariable("Phi[-180,180]");

Create a PDF from a RooFormulaVar via the workspace factory

In [4]:
toy.SetUp().LoadParameter("A[0.5,-1,1]");
toy.SetUp().FactoryPDF("EXPR::dist('1+A*cos(2*Phi/57.29578)',Phi,A)");
toy.SetUp().LoadSpeciesPDF("dist",100000);

In [5]:
ROOT.Here.Go(toy);

[#1] INFO:NumericIntegration -- RooRealIntegral::init(dist_Int[Phi]) using numeric integrator RooIntegrator1D to calculate Int(Phi)
[#1] INFO:NumericIntegration -- RooRealIntegral::init(dist_Int[Phi]) using numeric integrator RooIntegrator1D to calculate Int(Phi)


### Create a fitter which will just use 1 formula i.e. there will be no constant terms to cache

In [6]:
fit1 = ROOT.FitManager()
fit1.SetUp().SetOutDir("outOptimiseFit1/")

Create fit PDF with 2 parameters from sin and cos terms. Make sure initial values are not true values.

In [7]:
fit1.SetUp().LoadVariable("Phi[-180,180]");
fit1.SetUp().LoadParameter("A[0.,-1,1]");
fit1.SetUp().LoadParameter("B[0.5,-1,1]");
fit1.SetUp().LoadParameter("C[0.1,-1,1]");
fit1.SetUp().LoadParameter("D[-0.2,-1,1]");
fit1.SetUp().FactoryPDF("EXPR::dist1('1+A*cos(2*Phi/57.29578)+B*sin(2*Phi/57.29578)+C*cos(Phi/57.29578)+D*cos(4*Phi/57.29578)',Phi,A,B,C,D)");
fit1.SetUp().LoadSpeciesPDF("dist1",1);

Load the data created by the toy generator, arguments (treename, filename)

In [8]:
fit1.LoadData("ToyData","outOptimise/Toy0.root");

DataEvents::Load ToyData 1


Run the fit and time it

In [9]:
ROOT.gBenchmark.Start("fit1")
ROOT.Here.Go(fit1)
ROOT.gBenchmark.Stop("fit1")
ROOT.gBenchmark.Print("fit1")

[#0] WARNING:DataHandling -- RooDataSet::RooDataSet(DataEvents) WARNING: designated weight variable  not found in set of variables, no weighting will be assigned
FiledTree::~FiledTree()  tree name ToyData 100213 /work/Dropbox/HaSpect/dev/HASPECT6/tutorials/RooFitExamples/formulaFit/outOptimise/Toy0.root
RooDataSet::DataEvents[Phi,UID] = 100213 entries
[#1] INFO:NumericIntegration -- RooRealIntegral::init(dist1_Int[Phi]) using numeric integrator RooIntegrator1D to calculate Int(Phi)
[#1] INFO:Minization -- RooMinimizer::optimizeConst: activating const optimization
[#1] INFO:NumericIntegration -- RooRealIntegral::init(dist1_Int[Phi]) using numeric integrator RooIntegrator1D to calculate Int(Phi)
[#1] INFO:Minization --  The following expressions will be evaluated in cache-and-track mode: (dist1)
Minuit2Minimizer: Minimize with max-calls 2500 convergence for edm < 1 strategy 1
MnSeedGenerator: for initial parameters FCN = -456641.1502911
MnSeedGenerator: Initial state:   - FCN =  -456641.

Error in <TTree::SetBranchStatus>: unknown branch -> UID
Info in Minuit2Minimizer::Hesse : Hesse is valid - matrix is accurate
Info in Minuit2Minimizer::Hesse : Hesse is valid - matrix is accurate


### Create a fitter which will just use multiple components i.e. there will be constant terms to cache

In [10]:
fit2 = ROOT.FitManager()
fit2.SetUp().SetOutDir("outOptimiseFit2/")

Create fit PDF with 2 parameters from sin and cos terms. Make sure initial values are not true values.

In [11]:
fit2.SetUp().LoadVariable("Phi[-180,180]");
fit2.SetUp().LoadParameter("A[0.,-1,1]");
fit2.SetUp().LoadParameter("B[0.5,-1,1]");
fit2.SetUp().LoadParameter("C[0.1,-1,1]");
fit2.SetUp().LoadParameter("D[-0.2,-1,1]");
fit2.SetUp().LoadFormula("COS2=cos(2*@Phi[]/57.29578)");
fit2.SetUp().LoadFormula("COS=cos(@Phi[]/57.29578)");
fit2.SetUp().LoadFormula("COS4=cos(4*@Phi[]/57.29578)");
fit2.SetUp().LoadFormula("SIN2=sin(2*@Phi[]/57.29578)");
fit2.SetUp().FactoryPDF("EXPR::dist2('1+A*COS2+B*SIN2-C*COS+D*COS4'
                        ,Phi,A,B,C,D,COS2,COS,COS4,SIN2)");
fit2.SetUp().LoadSpeciesPDF("dist2",1);

[#1] INFO:ObjectHandling -- RooWorkspace::import() importing RooFormulaVar::COS2
[#1] INFO:ObjectHandling -- RooWorkspace::import() importing RooFormulaVar::COS
[#1] INFO:ObjectHandling -- RooWorkspace::import() importing RooFormulaVar::COS4
[#1] INFO:ObjectHandling -- RooWorkspace::import() importing RooFormulaVar::SIN2


Load the data created by the toy generator, arguments (treename, filename)

In [12]:
fit2.LoadData("ToyData","outOptimise/Toy0.root");

DataEvents::Load ToyData 1


Run the fit and time it

In [13]:
ROOT.gBenchmark.Start("fit1 unoptimised")
fit2.SetUp().AddFitOption(ROOT.RooFit.Optimize(0))
ROOT.Here.Go(fit2)
ROOT.gBenchmark.Stop("fit1 unoptimised")
ROOT.gBenchmark.Print("fit1 unoptimised")

[#1] INFO:ObjectHandling -- RooWorkspace::import() importing RooFormulaVar::COS2
[#1] INFO:ObjectHandling -- RooWorkspace::import() importing RooFormulaVar::COS
[#1] INFO:ObjectHandling -- RooWorkspace::import() importing RooFormulaVar::COS4
[#1] INFO:ObjectHandling -- RooWorkspace::import() importing RooFormulaVar::SIN2
[#0] WARNING:DataHandling -- RooDataSet::RooDataSet(DataEvents) WARNING: designated weight variable  not found in set of variables, no weighting will be assigned
FiledTree::~FiledTree()  tree name ToyData 100213 /work/Dropbox/HaSpect/dev/HASPECT6/tutorials/RooFitExamples/formulaFit/outOptimise/Toy0.root
RooDataSet::DataEvents[Phi,UID] = 100213 entries
[#1] INFO:NumericIntegration -- RooRealIntegral::init(dist2_Int[Phi]) using numeric integrator RooIntegrator1D to calculate Int(Phi)
Minuit2Minimizer: Minimize with max-calls 2500 convergence for edm < 1 strategy 1
MnSeedGenerator: for initial parameters FCN = -456517.1977043
MnSeedGenerator: Initial state:   - FCN =  -45

Error in <TTree::SetBranchStatus>: unknown branch -> UID
Info in Minuit2Minimizer::Hesse : Hesse is valid - matrix is accurate
Info in Minuit2Minimizer::Hesse : Hesse is valid - matrix is accurate
Warning in <TCanvas::Constructor>: Deleting canvas with same name: _Phi


In [14]:
ROOT.gBenchmark.Start("fit1 optimised")
fit2.SetUp().AddFitOption(ROOT.RooFit.Optimize(1))
ROOT.Here.Go(fit2)
ROOT.gBenchmark.Stop("fit1 optimised")
ROOT.gBenchmark.Print("fit1 optimised")

[#1] INFO:ObjectHandling -- RooWorkspace::import() importing RooFormulaVar::COS2
[#1] INFO:ObjectHandling -- RooWorkspace::import() importing RooFormulaVar::COS
[#1] INFO:ObjectHandling -- RooWorkspace::import() importing RooFormulaVar::COS4
[#1] INFO:ObjectHandling -- RooWorkspace::import() importing RooFormulaVar::SIN2
[#0] WARNING:DataHandling -- RooDataSet::RooDataSet(DataEvents) WARNING: designated weight variable  not found in set of variables, no weighting will be assigned
FiledTree::~FiledTree()  tree name ToyData 100213 /work/Dropbox/HaSpect/dev/HASPECT6/tutorials/RooFitExamples/formulaFit/outOptimise/Toy0.root
RooDataSet::DataEvents[Phi,UID] = 100213 entries
[#1] INFO:NumericIntegration -- RooRealIntegral::init(dist2_Int[Phi]) using numeric integrator RooIntegrator1D to calculate Int(Phi)
[#1] INFO:Minization -- RooMinimizer::optimizeConst: activating const optimization
[#1] INFO:Minization --  The following expressions have been identified as constant and will be precalculat

Error in <TTree::SetBranchStatus>: unknown branch -> UID
Info in Minuit2Minimizer::Hesse : Hesse is valid - matrix is accurate
Info in Minuit2Minimizer::Hesse : Hesse is valid - matrix is accurate
Warning in <TCanvas::Constructor>: Deleting canvas with same name: _Phi


### Create a fitter which will just use multiple components with ComponentsPDF

First need to construct flat MC data for calculating normalisation integral. Here we generate 100000 events.

In [15]:
flat = ROOT.ToyManager(1)
flat.SetUp().SetOutDir("outOptimiseFlat/")
flat.SetUp().LoadVariable("Phi[-180,180]");
flat.SetUp().FactoryPDF("EXPR::flat('1',Phi)");
flat.SetUp().LoadSpeciesPDF("flat",100000);
ROOT.Here.Go(flat);

[#1] INFO:NumericIntegration -- RooRealIntegral::init(flat_Int[Phi]) using numeric integrator RooIntegrator1D to calculate Int(Phi)
[#1] INFO:NumericIntegration -- RooRealIntegral::init(flat_Int[Phi]) using numeric integrator RooIntegrator1D to calculate Int(Phi)


Construct another FitManager, this time for the ComponentsPDF

In [16]:
fit3 = ROOT.FitManager()
fit3.SetUp().SetOutDir("outOptimiseFit3/")

In [17]:
fit3.SetUp().LoadVariable("Phi[-180,180]");
fit3.SetUp().LoadParameter("A[0.,-1,1]");
fit3.SetUp().LoadParameter("B[0.5,-1,1]");
fit3.SetUp().LoadParameter("C[0.1,-1,1]");
fit3.SetUp().LoadParameter("D[-0.2,-1,1]");
fit3.SetUp().LoadFormula("COS2=cos(2*@Phi[]/57.29578)");
fit3.SetUp().LoadFormula("COS=cos(@Phi[]/57.29578)");
fit3.SetUp().LoadFormula("COS4=cos(4*@Phi[]/57.29578)");
fit3.SetUp().LoadFormula("SIN2=sin(2*@Phi[]/57.29578)");

[#1] INFO:ObjectHandling -- RooWorkspace::import() importing RooFormulaVar::COS2
[#1] INFO:ObjectHandling -- RooWorkspace::import() importing RooFormulaVar::COS
[#1] INFO:ObjectHandling -- RooWorkspace::import() importing RooFormulaVar::COS4
[#1] INFO:ObjectHandling -- RooWorkspace::import() importing RooFormulaVar::SIN2


Create a RooComponentsPDF as 1 + A*COS2 + B*SIN2 + C*COS + D*COS4

In [18]:
fit3.SetUp().FactoryPDF("RooComponentsPDF::dist3
                        '(1,{Phi},=A;COS2:B;SIN2:C;COS:D;COS4)");
fit3.SetUp().LoadSpeciesPDF("dist3",1)

[#1] INFO:ObjectHandling -- RooWorkspace::import() importing HS::FIT::RooComponentsPDF::dist3


In [19]:
fit3.LoadData("ToyData","outOptimise/Toy0.root");
fit3.LoadSimulated("ToyData","outOptimiseFlat/Toy0.root","dist3");

DataEvents::Load ToyData 1


Run the fit and time it

In [21]:
ROOT.gBenchmark.Reset()
ROOT.gBenchmark.Start("fit3")
fit3.SetUp().AddFitOption(ROOT.RooFit.Optimize(1))
ROOT.Here.Go(fit3)
ROOT.gBenchmark.Stop("fit3")
ROOT.gBenchmark.Print("fit3")

fit3      : Real Time =   2.24 seconds Cpu Time =   2.16 seconds


Error in <TTree::SetBranchStatus>: unknown branch -> UID
Info in <HS::FIT::RooComponentsPDF::RooHSEventsPDF::SetEvTree>:  with name ToyData and cut 
Info in <HS::FIT::RooComponentsPDF::RooHSEventsPDF::AddProtoData>: Added data branch UID
Info in Minuit2Minimizer::Hesse : Hesse is valid - matrix is accurate
Info in Minuit2Minimizer::Hesse : Hesse is valid - matrix is accurate
Warning in <TCanvas::Constructor>: Deleting canvas with same name: _Phi
Warning in <TROOT::Append>: Replacing existing TH1: Phi (Potential memory leak).
